# Topic 4: Loops and Cycles

Master iterative workflows by creating loops and cycles in your graphs. Learn how to build self-improving systems that can iterate until a condition is met.

## Learning Objectives

- Create cyclic graphs with loops
- Implement iteration limits to prevent infinite loops
- Build self-correcting workflows
- Use state to control loop termination

In [ ]:
# Setup
import os
import getpass
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(model="claude-sonnet-4-20250514")
print("✓ Setup complete!")

## Understanding Loops in LangGraph

Loops allow a graph to revisit nodes multiple times until a condition is met. This is perfect for:
- Iterative refinement
- Self-correction
- Retry logic
- Progressive improvement

## Example 1: Code Review Loop

Let's build a system that iteratively reviews and improves code until it meets quality standards.

In [ ]:
# Define our state
class CodeReviewState(TypedDict):
    code: str
    review_feedback: str
    quality_score: int
    iteration: int
    max_iterations: int

print("✓ CodeReviewState defined")

In [ ]:
def review_code(state: CodeReviewState) -> CodeReviewState:
    """Review code and provide quality score and feedback."""
    print(f"\n🔍 Review iteration {state['iteration'] + 1}...")
    
    prompt = f"""Review this Python code and rate its quality from 1-10:

CODE:
```python
{state['code']}
```

Provide:
1. Quality Score (1-10)
2. Specific improvements needed
3. Best practices violations

Format your response as:
Score: [number]
Feedback: [detailed feedback]"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    content = response.content
    
    # Extract score
    score = 5
    for line in content.split('\n'):
        if 'Score:' in line or 'score:' in line:
            try:
                score = int(''.join(filter(str.isdigit, line)))
                break
            except:
                pass
    
    print(f"   Quality Score: {score}/10")
    
    return {
        "review_feedback": content,
        "quality_score": score,
        "iteration": state['iteration'] + 1
    }

def improve_code(state: CodeReviewState) -> CodeReviewState:
    """Improve code based on review feedback."""
    print("✨ Improving code based on feedback...")
    
    prompt = f"""Improve this code based on the review feedback:

CURRENT CODE:
```python
{state['code']}
```

REVIEW FEEDBACK:
{state['review_feedback']}

Provide ONLY the improved code, no explanations."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    # Extract code from response
    improved_code = response.content
    if '```python' in improved_code:
        improved_code = improved_code.split('```python')[1].split('```')[0].strip()
    elif '```' in improved_code:
        improved_code = improved_code.split('```')[1].split('```')[0].strip()
    
    return {"code": improved_code}

print("✓ Review and improve nodes created")

## The Key: Conditional Routing for Loops

The router function decides whether to continue looping or exit:

In [ ]:
def should_continue(state: CodeReviewState) -> Literal["improve", "end"]:
    """Decide whether to continue improving or finish."""
    
    # Stop if quality is good enough
    if state['quality_score'] >= 8:
        print("✅ Quality threshold met! Finishing...")
        return "end"
    
    # Stop if max iterations reached
    if state['iteration'] >= state['max_iterations']:
        print("⚠️  Max iterations reached. Finishing...")
        return "end"
    
    # Continue improving
    print("🔄 Continuing iteration...")
    return "improve"

print("✓ Router function created")

## Building the Cyclic Graph

Notice how we create a loop by routing back to the review node:

In [ ]:
# Create the graph
graph_builder = StateGraph(CodeReviewState)

# Add nodes
graph_builder.add_node("review", review_code)
graph_builder.add_node("improve", improve_code)

# Start with review
graph_builder.add_edge(START, "review")

# After review, decide whether to continue or end
graph_builder.add_conditional_edges(
    "review",
    should_continue,
    {
        "improve": "improve",
        "end": END
    }
)

# After improving, loop back to review - THIS CREATES THE CYCLE!
graph_builder.add_edge("improve", "review")

# Compile
code_review_graph = graph_builder.compile()

print("✓ Cyclic code review graph compiled!")

## Visualize the Loop

In [ ]:
from IPython.display import Image, display

try:
    display(Image(code_review_graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph structure:")
    print("START -> review -> {improve -> review (loop)} -> END")

## Test the Loop

Let's see the iterative improvement in action:

In [ ]:
# Sample code that needs improvement
initial_code = """def calculate(x, y):
    return x + y"""

print("Initial Code:")
print(initial_code)
print("\n" + "="*60)

# Run the iterative improvement
result = code_review_graph.invoke({
    "code": initial_code,
    "review_feedback": "",
    "quality_score": 0,
    "iteration": 0,
    "max_iterations": 3
})

print("\n" + "="*60)
print("Final Results:")
print("="*60)
print(f"\nIterations: {result['iteration']}")
print(f"Final Quality Score: {result['quality_score']}/10")
print(f"\nImproved Code:\n{result['code']}")

## Example 2: Research Paper Refinement

Let's build another loop for iteratively refining research findings:

In [ ]:
class ResearchState(TypedDict):
    topic: str
    findings: str
    critique: str
    completeness_score: int
    iteration: int
    max_iterations: int

def research_topic(state: ResearchState) -> ResearchState:
    """Research the topic and compile findings."""
    print(f"\n🔬 Research iteration {state['iteration'] + 1}...")
    
    if state['iteration'] == 0:
        prompt = f"Research this topic and provide key findings: {state['topic']}"
    else:
        prompt = f"""Expand your research based on this critique:
        
CURRENT FINDINGS:
{state['findings']}

CRITIQUE:
{state['critique']}

Provide improved and expanded findings."""
    
    response = model.invoke([HumanMessage(content=prompt)])
    
    return {
        "findings": response.content,
        "iteration": state['iteration'] + 1
    }

def critique_research(state: ResearchState) -> ResearchState:
    """Critique the research for completeness."""
    print("🔍 Critiquing research...")
    
    prompt = f"""Rate these research findings for completeness (1-10) and provide critique:

{state['findings']}

Format:
Score: [1-10]
Critique: [what's missing or needs improvement]"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    content = response.content
    
    # Extract score
    score = 5
    for line in content.split('\n'):
        if 'Score:' in line or 'score:' in line:
            try:
                score = int(''.join(filter(str.isdigit, line)))
                break
            except:
                pass
    
    print(f"   Completeness Score: {score}/10")
    
    return {
        "critique": content,
        "completeness_score": score
    }

def should_continue_research(state: ResearchState) -> Literal["research", "end"]:
    """Decide whether to continue researching."""
    if state['completeness_score'] >= 8:
        print("✅ Research is complete!")
        return "end"
    if state['iteration'] >= state['max_iterations']:
        print("⚠️  Max iterations reached.")
        return "end"
    print("🔄 Continuing research...")
    return "research"

print("✓ Research nodes and router created")

In [ ]:
# Build research graph
research_graph_builder = StateGraph(ResearchState)

research_graph_builder.add_node("research", research_topic)
research_graph_builder.add_node("critique", critique_research)

research_graph_builder.add_edge(START, "research")
research_graph_builder.add_edge("research", "critique")

# The conditional edge that creates the loop
research_graph_builder.add_conditional_edges(
    "critique",
    should_continue_research,
    {
        "research": "research",
        "end": END
    }
)

research_graph = research_graph_builder.compile()

print("✓ Research refinement graph compiled!")

In [ ]:
# Test the research loop
print("Starting iterative research process...")
print("="*60)

result = research_graph.invoke({
    "topic": "The impact of AI on software development",
    "findings": "",
    "critique": "",
    "completeness_score": 0,
    "iteration": 0,
    "max_iterations": 2
})

print("\n" + "="*60)
print("Research Complete!")
print("="*60)
print(f"\nIterations: {result['iteration']}")
print(f"Completeness Score: {result['completeness_score']}/10")
print(f"\nFinal Findings:\n{result['findings']}")

## Key Patterns for Loops

When building loops in LangGraph, always:

1. **Include iteration tracking** in your state
2. **Set max iterations** to prevent infinite loops
3. **Define clear exit conditions** (quality thresholds, completion criteria)
4. **Use conditional edges** to control loop flow
5. **Add progress logging** to track iterations

## Exercise: Build a Self-Correcting Translator

Create a translation system that:
1. Translates text
2. Back-translates to check accuracy
3. Improves translation if accuracy is low
4. Loops until accuracy is high or max iterations reached

Hint: Your state should track the original text, translation, back-translation, accuracy score, and iteration count.

In [ ]:
# Your code here!

class TranslationState(TypedDict):
    original: str
    translation: str
    back_translation: str
    accuracy_score: int
    iteration: int
    max_iterations: int

# TODO: Define translate node
# TODO: Define back_translate node
# TODO: Define evaluate_accuracy node
# TODO: Define router function
# TODO: Build graph with loop
# TODO: Test it

## Key Takeaways

In this notebook, you learned:

1. ✅ How to create cyclic graphs with loops
2. ✅ Using iteration tracking and limits to control loops
3. ✅ Building self-correcting and iterative improvement systems
4. ✅ Implementing conditional routing for loop control
5. ✅ Creating exit conditions based on quality or iteration limits

## Next Steps

Continue to **Topic 5: Human-in-the-Loop** to learn how to add human approval and interaction to your workflows!